In [ ]:
import numpy as np
from random import choices, randrange
import random
import matplotlib.pyplot as plt
from numba import njit
from sklearn.datasets import make_blobs
from utility import *
from tqdm import tqdm
from sklearn.cluster import k_means

Implementazione dell'euristica di Simulated Annealing

In [ ]:
def change_sol(sol, K):
    new_sol = sol.copy()
    n = randrange(len(new_sol))#ritorna un numero casualmente nel range (0, len(new_sol))
    old_value = new_sol[n]

    while old_value == new_sol[n]:
        new_sol[n] = randrange(K)

    return new_sol

def simulated_annealing(base_sol, points, K, iters, alpha, verbose = True):
    best_sol = base_sol
    base_val = squared_inner_distance(best_sol, points, K)
    best_value = base_val
    T = base_val*1/100
    Tf = 10**-5
    curr_sol = base_sol
    curr_val = best_value
    iter = 1

    old_sol = base_sol
    finito = False
    no_update = 0

    while(finito == False):
        if(verbose):
            print("Iteration number:", iter, "Best value percentuale: ", curr_val/base_val*100, "% T:", T)
        iter = 1 + iter
        old_sol = curr_sol
        
        for i in range(iters):
            candidate = change_sol(curr_sol, K)
            val_candidate = squared_inner_distance(candidate, points, K)

            if(val_candidate < best_value):
                best_value = val_candidate#viene memorizzato il best value, perchè potrei perderlo nelle iterazioni successive
                best_sol = candidate
            
            if(val_candidate < curr_val):
                curr_val = val_candidate
                curr_sol = candidate
                no_update = 0
            else:
                r = random.uniform(0, 1)
                delta = abs(curr_val - val_candidate)
                tresh = np.exp(-delta/T)
                no_update = 0
                if(r < tresh):
                    curr_val = val_candidate
                    curr_sol = candidate
                else:
                    no_update = no_update + 1

        T = alpha*T
    
        if(no_update == 500 or T <= Tf):
            finito = True
    return best_sol

Creazione di un'istanza di testing con 1000 punti nello spazio R^2 e 5 clusters e testing dell'euristica

In [ ]:
points, centroids = make_blobs(n_samples=1000, centers=5, n_features=2, random_state=2)
N = len(points)
K = 5

sol = create_initial_sol(points, K)
sol = simulated_annealing(sol, points, K, 10, 0.99)

print("{:.5E}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)

In [ ]:
n_points = [500,1000,1000,1000,1000,1000,1500,1500,2000,3000,5000,10000]
n_clusters = [5,2,4,5,6,7,5,10,5,5,5]
dim_points = [32,32,32,32,20,20,18,18,18,16,16,16]
vals = []

for test in tqdm(range(1,13)):
    points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{test}.txt')
    N = len(points)
    K = n_clusters[test-1]

    sol = create_initial_sol(points, K)
    sol = simulated_annealing(sol, points, K, 10, 0.99, False)
    val = squared_inner_distance(sol, points, K)
    vals.append(val)


with open("risultatiSA.txt", 'w') as file:
    file.write("K-means:\n")
    file.write(str(vals))

In [ ]:
print(vals)